In [4]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col
import numpy as np

x1 = np.random.rand(1, 3, 7, 7)  # batch_size, channel_size, height, width
coll = im2col(x1, 5, 5, stride=1, pad=0)
print(coll.shape)

x2 = np.random.rand(10, 3, 7, 7)  # batch_size, channel_size, height, width
coll = im2col(x2, 5, 5, stride=1, pad=0)
print(coll.shape)


(9, 75)
(90, 75)
